In [1]:

import railtracks as rt
from dotenv import load_dotenv
from typing import Literal
from pydantic import BaseModel, Field

_ = load_dotenv()  # take environment variables from .env.

In [ ]:
RESPONDER_SYSTEM_MESSAGE = ""

CRITIQUER_SYSTEM_MESSAGE = ""

Of course we need to set up some tools for our agents to use 

In [ ]:
class FeedbackEntry(BaseModel):
    section_of_interest: str | None
    feedback: str
    feedback_type: Literal["positive", "negative", "neutral"]
    identifier: int
    action_taken: str | None = None
    accepted: bool | None = None

    def present_to_llm(self):
        return f"Feedback Type: {self.feedback_type}\nSection of Interest: {self.section_of_interest}\nFeedback: {self.feedback}\nAction Taken: {self.action_taken}\nAccepted: {self.accepted}"


@rt.function_node
async def add_feedback(text_ref: str | None, feedback: str, feedback_type: Literal["positive", "negative", "neutral"]) -> str:
    """
    Adds feedback to the feedback log for the current entry.
    
    Args:
        text_ref (str | None): The section of the entry the feedback is about. If general feedback, this can be None.
        feedback (str): The feedback you would like to provide.
        feedback_type (Literal["positive", "negative", "neutral"]): The type of feedback.
    """
    entry_id: str = rt.context.get("entry_id")
    feedback_log: list[FeedbackEntry] = rt.context.get("feedback_log")[entry_id]

    if text_ref not in rt.context.get("current_entry")[entry_id]:
        raise ValueError(f"Section {text_ref} not found in current entry.")
    
    current_entry = rt.context.get("feedback_counter")

    feedback_log.append(FeedbackEntry(section_of_interest=text_ref, feedback=feedback, feedback_type=feedback_type, identifier=current_entry))
    current_entry += 1

    rt.context.put("feedback_counter", current_entry)


    return "Feedback Submitted Successfully."


@rt.function_node
async def complete_feedback(feedback_id: int, action_taken: str, accepted: bool):
    feedback_log: list[FeedbackEntry] = rt.context.get("feedback_log")

    id_list = [x.identifier for x in feedback_log]


    if feedback_id not in id_list:
        raise ValueError("Your feedback identifier does not exist")
    feedback = None
    for i in id_list:
        if feedback_id == i:
            feedback = feedback_log[i]
            break

    assert feedback is not None

    if feedback.accepted is not None:
        raise ValueError("Action has already been taken on this feedback. ")

    feedback.action_taken = action_taken
    feedback.accepted = accepted

    return f"You succesfully have closed the feedback:\n{feedback.feedback}\nWith resolution: {feedback.accepted} - {feedback.action_taken}"



In [ ]:
Responder = rt.agent_node(
    name="Responder",
    system_message=RESPONDER_SYSTEM_MESSAGE,
    llm=rt.llm.OpenAILLM('gpt-4o'),

)

Critique = rt.agent_node(
    name="Critique",
    system_message="You are an expert at critiquing answers for correctness and completeness.",
    llm=rt.llm.OpenAILLM('gpt-5'),
)




In [ ]:
feedback_log = []